# Creating a Deep Model to predict the antidepressant effect

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt # For data viz
import pandas as pd
import numpy as np
import sys

print('System Version:', sys.version)
print('PyTorch version', torch.__version__)
print('Numpy version', np.__version__)
print('Pandas version', pd.__version__)

In [ ]:
# Confirm device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")
try: 
    name = torch.cuda.get_device_name(0)
    count = torch.cuda.device_count()
    print(f"Device count: {count}")
    print(f"Device name: {name}")
except RuntimeError:
    print('No GPUs detected')

In [ ]:
# Load the data
X_train = np.load('data/X_TRAIN_RAW.npy')
X_test = np.load('data/X_TEST_RAW.npy')

y_train = np.load('data/y_TRAIN_RAW.npy')
y_test = np.load('data/y_TEST_RAW.npy')


In [ ]:
# Define the dataset
class COPEDataset(Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        cope_data = self.data[index]
        cope_data = (cope_data - np.min(cope_data)) / (np.max(cope_data) - np.min(cope_data))  # normalize

        label = self.target[index]
        volume = torch.tensor(cope_data, dtype=torch.float32).unsqueeze(0)  # (1, 91, 109, 91)
        label = torch.tensor([1.0, 0.0] if label == 0 else [0.0, 1.0], dtype=torch.float32)
        return volume, label

In [ ]:
# Initiate the dataset and data loader
train_dataset = COPEDataset(X_train, y_train)
test_dataset = COPEDataset(X_test, y_test)
train_dataloader = DataLoader(train_dataset, batch_size=10)
test_dataloader = DataLoader(test_dataset)

In [ ]:
# Define the model
class BrainClassifier3D(nn.Module):
    def __init__(self):
        super(BrainClassifier3D, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv3d(1, 16, kernel_size=3, stride=2, padding=1),   # volume (16, 45, 54, 45)
            nn.BatchNorm3d(16),
            nn.ReLU(),

            nn.Conv3d(16, 32, kernel_size=3, stride=2, padding=1),  # volume (32, 22, 26, 22)
            nn.BatchNorm3d(32),
            nn.ReLU(),

            # nn.Conv3d(32, 64, kernel_size=3, stride=2, padding=1),  #  (64, 12, 10, 12)
            # nn.BatchNorm3d(64),
            # nn.ReLU()
        )

        # Calculate the size of the flattened layer dynamically
        # We run a dummy tensor through the conv/pool layers to find the shape
        self._to_linear = None
        self._get_conv_output((1, 23, 19, 15)) # Dummy run to calculate self._to_linear


        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Linear(self._to_linear, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 2),  # 2 output classes
            nn.Softmax()
        )

    def _get_conv_output(self, shape):
        """Helper function to calculate the input size for the fully connected layers."""
        # Create a dummy input tensor with batch size 1
        dummy_input = torch.rand(1, *shape) 
        output_features = self.conv_layers(dummy_input)
        # Store the calculated size
        self._to_linear = output_features.view(output_features.size(0), -1).size(1)

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x  # logits


In [ ]:
# Setup
model = BrainClassifier3D().cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# Train
for epoch in range(20):
    model.train()
    total_loss, correct = 0.0, 0

    for batch in train_dataloader:
        inputs, labels = batch
        inputs, labels = inputs.cuda(), labels.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == torch.argmax(labels, dim=1)).sum().item()

    acc = correct / len(train_dataloader.dataset)
    print(f"Epoch {epoch+1}, Loss: {total_loss:.2f}, Accuracy: {acc:.4f}")


In [ ]:
# Test the model

def evaluate(model, dataloader):
    model.eval()
    correct = 0
    preds_list = np.array([])
    actual_list = np.array([])
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            preds_list = np.append(preds_list, preds.item())
            actual_list = np.append(actual_list, torch.argmax(labels, dim=1).item())
            print(f'Predicted: {preds.item()}   True: {torch.argmax(labels, dim=1).item()}')
            correct += (preds == torch.argmax(labels, dim=1)).sum().item()
    return correct / len(dataloader.dataset), preds_list, actual_list

val_acc, preds_list, actual_list = evaluate(model, test_dataloader)
print(f"Validation Accuracy: {val_acc:.4f}")




In [ ]:
reg_vals = np.load('data/y_TEST_RAW_REG.npy')
thresh = 8

for i in range(len(reg_vals)):
    if preds_list[i] == 0:
        plt.plot((i+1), reg_vals[i], 'o', color='red')
    else:
        plt.plot((i+1), reg_vals[i], 'o', color='green')

plt.axhline(y=thresh, color='grey', linestyle='--', linewidth=2, label=f'Threshold at y={thresh}')

# Add labels and title
plt.xlabel("Subjects")
plt.ylabel("Mood Score Change WR -> SD")
plt.title("Predictive Results")